In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access specific file for specific participant
def load_epochs(pp_num, root_dir, erp_path):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    if erp_path == 'frn':
        epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))
    elif erp_path == 'lrp':
        epochs = epochs.apply_baseline(baseline = (-1.3, -1.0))
    elif erp_path == 'ern':
        epochs = epochs.apply_baseline(baseline = (-1.35, -1.05))

    return epochs, pp_directory

In [3]:
#get index of trials from behavioral data
#rdm before rot and rdm before mir are read in as two different behavioral data
def epoch_index(task, listval, pp, target = np.nan):
    if task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
    elif task == 'rdmrot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees.csv')
    elif task == 'rdmmir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMMIR_learningcurve_degrees.csv')
    elif task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
    elif task == 'mir':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\MIR_learningcurve_degrees.csv')
   
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata) if listval[0] <= i < listval[1]]
    
    #epochs_out = epochs[index]
    numtrials = len(index)
    
    #return epochs_out, numtrials
    return index, numtrials

In [4]:
#separate function to get indexed epochs for random, since we have them before rotation or mirror
def trial_list_rdm(task, pp, index):
    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rdmrot':
            #get trial after first task (trial 48 to 95, 234 to 281)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
                    
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
            
    elif pp % 2 == 1:   
        if task == 'rdmrot':
            #get trial after first five tasks (trial 234 to 281)
            tasksdf = df[0:282]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16153]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            
            
        elif task == 'rdmmir':
            #get trial after first task (trial 48 to 95)
            tasksdf = df[0:96]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16150]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)
            

    
    return(triallist)
    

In [5]:
#get matching trials from eeg data, where we account for missing triggers
def epoch_trials(task, pp, epochs, index):

    df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\eeg\p%03d\%s\mra_p%03d_tasktrigindex.csv' % (pp, erps, pp))
    
    if task == 'aln':
        #get trial numbers where tasktrig is 16149 and trigidx is 1
        tasksdf = df[0:48]
        subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16149]
        triallist = []
        for i in index:
            ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
            nint = int(ndat['trigidx'])
            if nint == 1:
                trial = int(ndat['ctrial'])
                triallist.append(trial)

        epochs = epochs[triallist]
    
    #for the other tasks, the order depends on their pp number
    #even pp numbers would experience rotation first (need random before this)
    #need to also take into account missing triggers from previous tasks
    if pp % 2 == 0:
        if task == 'rot':
            #rot trials are after first two tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first task (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            tasksdf = df[0:372]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16151]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
            
        elif task == 'mir':
            #get trial after first five tasks (trial 96 to 185)
            tasksdf = df[0:186]
            subtasksdf = tasksdf.loc[tasksdf['tasktrig'] == 16154]
            triallist = []
            for i in index:
                ndat = subtasksdf.loc[subtasksdf['trialno'] == i]
                nint = int(ndat['trigidx'])
                if nint == 1:
                    trial = int(ndat['ctrial'])
                    triallist.append(trial)

            epochs = epochs[triallist]
    
    return(epochs)

In [6]:
#save epochs
def output_condition_epochs(pp_num, data, ppdir, condname, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-epo.fif' % (pp_num, condname, task))
    data.save(out_fname, overwrite = True)

In [7]:
#save as an evoked object
def output_condition_evoked(pp_num, data, ppdir, condname, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-ave.fif' % (pp_num, condname, task))
    evoked.save(out_fname, overwrite = True)

In [ ]:
#save as evoked objects - runs for any erp type
#each perturbation condition can have separate loops

#lowpass filter cutoff
l_cutoff = 30

#ALIGNED
#have one condition for aligned
conditions = [0,48]

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    idx, numtrials = epoch_index('aln', conditions, pp)
    #print(idx)
    aldata = epoch_trials('aln', pp, data, idx)
    #print(aldata)
    output_condition_epochs(pp, aldata, ppdir, 'early_late', 'aligned')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'early_late', 'aligned')

#ROTATED
#separate early and late for perturbed conditions as below:
conditions = [[0,45], [45,90]]
conditionnames = ['early', 'late']

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        idx, numtrials = epoch_index('rot', conditions[cond], pp)
        pdata = epoch_trials('rot', pp, data, idx)
        output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rot')
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rot')
        
#MIRROR
#separate early and late for perturbed conditions as below:
conditions = [[0,45], [45,90]]
conditionnames = ['early', 'late']

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        idx, numtrials = epoch_index('mir', conditions[cond], pp)
        pdata = epoch_trials('mir', pp, data, idx)
        output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'mir')
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'mir')
        
#RANDOM 
#combine rdm rot and rdm mir into one per participant, but get correct indices
conditions = [[0,24], [24,48]]
conditionnames = ['early', 'late']

for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root, erp_path = erps)
    for cond in range(0, len(conditions)):
        idx, numtrials = epoch_index('rdmrot', conditions[cond], pp)
        rdmrot_trials = trial_list_rdm('rdmrot', pp, idx)
        #print(rdmrot_trials)
        idx, numtrials = epoch_index('rdmmir', conditions[cond], pp)
        rdmmir_trials = trial_list_rdm('rdmmir', pp, idx)
        #print(rdmmir_trials)
        if cond == 0:
            rdm_early_trials = rdmrot_trials + rdmmir_trials
            rdm_early_trials.sort()
            pdata = data[rdm_early_trials]
        elif cond == 1:
            rdm_late_trials = rdmrot_trials + rdmmir_trials
            rdm_late_trials.sort()
            pdata = data[rdm_late_trials]
        output_condition_epochs(pp, pdata, ppdir, conditionnames[cond], 'rdm')
        #filter at 30 Hz
        pdata = pdata.filter(l_freq= None, h_freq=l_cutoff)
        #then average across trials
        output_condition_evoked(pp, pdata, ppdir, conditionnames[cond], 'rdm')

In [8]:
#get grand averages - split according to early or late conditions
root_directory = root
data_directory = os.path.join(root_directory, 'data/eeg/')

#read in evoked object
tasknames = ['rot', 'rdm', 'mir']
conditionnames = ['early', 'late']

grand_aligned = []
grand_early = []
grand_late = []

#aligned
flist = []
evoked_list = []

for pp in participants:
    id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    pp_directory = os.path.join(id_directory, erps)
    fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], 'early_late', 'aligned'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
        
grand_aligned = mne.grand_average(evoked_list) 
aligned_flist = evoked_list
    
#rot and rdm and mir
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        flist = []
        evoked_list = []
        for pp in participants:
            id_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            pp_directory = os.path.join(id_directory, erps)
            fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = flist[pp][0]
            evoked_list.append(evoked)
        if task == 0 and condition == 0:
            grand_rot_early = mne.grand_average(evoked_list)
            early_rot_flist = evoked_list
        elif task == 0 and condition == 1:
            grand_rot_late = mne.grand_average(evoked_list)
            late_rot_flist = evoked_list
        elif task == 1 and condition == 0:
            grand_rdm_early = mne.grand_average(evoked_list)
            early_rdm_flist = evoked_list
        elif task == 1 and condition == 1:
            grand_rdm_late = mne.grand_average(evoked_list)
            late_rdm_flist = evoked_list
        elif task == 2 and condition == 0:
            grand_mir_early = mne.grand_average(evoked_list)
            early_mir_flist = evoked_list
        elif task == 2 and condition == 1:
            grand_mir_late = mne.grand_average(evoked_list)
            late_mir_flist = evoked_list
            

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p000\frn\p000_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p020\frn\p020_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_early_late_aligned-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_early_late_aligned-ave.fif ...


No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p008\frn\p008_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p009\frn\p009_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirR

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p028\frn\p028_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p029\frn\p029_early_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_early_rot-ave.fif ...
    Found the data of inter

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p016\frn\p016_late_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p017\frn\p017_late_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p018\frn\p018_late_rot-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compe

...

        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p013\frn\p013_late_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p014\frn\p014_late_rdm-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 48 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] se

Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p001\frn\p001_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p002\frn\p002_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p003\frn\p003_early_mir-ave.fif ...
    Found the data of inter

Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p021\frn\p021_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p022\frn\p022_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p023\frn\p023_early_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF co

    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p010\frn\p010_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p011\frn\p011_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified 

        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p030\frn\p030_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Documents\Science\MirRevAdaptEEG\data\eeg\p031\frn\p031_late_mir-ave.fif ...
    Found the data of interest:
        t =   -2000.00 ...    1995.00 ms (16142)
        0 CTF compensation matrices available
        nave = 45 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is

In [ ]:
#convert evokeds to data frame, so we can plot the ERPs better - runs for FRN/ERN ONLY

flists = ['aln', 'earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aligned_flist[pp].to_data_frame()
        elif f == 'earlyrot':
            df = early_rot_flist[pp].to_data_frame()
        elif f == 'laterot':
            df = late_rot_flist[pp].to_data_frame()
        elif f == 'earlyrdm':
            df = early_rdm_flist[pp].to_data_frame()
        elif f == 'laterdm':
            df = late_rdm_flist[pp].to_data_frame()
        elif f == 'earlymir':
            df = early_mir_flist[pp].to_data_frame()
        elif f == 'latemir':
            df = late_mir_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        #df = df[['C3', 'C4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s.csv' % (f, erps))
    newdf.to_csv(newdf_filename)
        

In [9]:
#plot FRN/ERN here using MNE functions
#channels = ['FCz', 'Fz']
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']
#channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               early_rot = early_rot_flist,
               late_rot = late_rot_flist, 
               early_rdm = early_rdm_flist, 
               late_rdm = late_rdm_flist,
               early_mir = early_mir_flist,
               late_mir = late_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

More than 6 channels, truncating title ...
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 1 Axes>]

In [10]:
#calculate difference waves for FRN/ERN
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rot'][pp]], 
                                                            weights=[-1, 1])
            earlyrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rot'][pp]], 
                                                         weights=[-1, 1])
            laterot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rdm'][pp]], 
                                                         weights=[-1, 1])
            earlyrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rdm'][pp]], 
                                                         weights=[-1, 1])
            laterdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_mir'][pp]], 
                                                         weights=[-1, 1])
            earlymir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_mir'][pp]], 
                                                         weights=[-1, 1])
            latemir_diff.append(diffevks)


In [ ]:
#saved difference waves as data frame for FRN/ERN

difflists = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'earlyrot':
            df = earlyrot_diff[pp].to_data_frame()
        elif diff == 'laterot':
            df = laterot_diff[pp].to_data_frame()
        elif diff == 'earlyrdm':
            df = earlyrdm_diff[pp].to_data_frame()
        elif diff == 'laterdm':
            df = laterdm_diff[pp].to_data_frame()
        elif diff == 'earlymir':
            df = earlymir_diff[pp].to_data_frame()
        elif diff == 'latemir':
            df = latemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_%s_%s.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        


In [11]:
#plot difference waves for FRN/ERN here in MNE
mne.viz.plot_compare_evokeds({'Earlyrot - Aligned':earlyrot_diff, 'Laterot - Aligned':laterot_diff, 'Earlyrdm - Aligned':earlyrdm_diff, 'Laterdm - Aligned':laterdm_diff, 'Earlymir - Aligned':earlymir_diff, 'Latemir - Aligned':latemir_diff,}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');

combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


In [12]:
#calculate difference waves for FRN/ERN - LATE MINUS EARLY
diffconds = ['rot_diff', 'rdm_diff', 'mir_diff']
rot_diff = []
rdm_diff = []
mir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['early_rot'][pp], evokeds['late_rot'][pp]], 
                                                            weights=[-1, 1])
            rot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['early_rdm'][pp], evokeds['late_rdm'][pp]], 
                                                         weights=[-1, 1])
            rdm_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['early_mir'][pp], evokeds['late_mir'][pp]], 
                                                         weights=[-1, 1])
            mir_diff.append(diffevks)

In [13]:
#saved EARLY VS LATE difference waves as data frame for FRN/ERN

difflists = ['rot_diff', 'rdm_diff', 'mir_diff']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'rot_diff':
            df = rot_diff[pp].to_data_frame()
        elif diff == 'rdm_diff':
            df = rdm_diff[pp].to_data_frame()
        elif diff == 'mir_diff':
            df = mir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_EvL_%s_%s.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)

In [14]:
#plot EARLY VS LATE difference waves for FRN/ERN here in MNE
mne.viz.plot_compare_evokeds({'Rot':rot_diff, 'Rdm':rdm_diff, 'Mir':mir_diff}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Early Vs Late Difference Wave');

combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


In [ ]:
#runs for LRP ONLY: need separate files for C3 and C4

flists = ['aln', 'earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
channels = ['C3', 'C4']
for channel in channels:
    for f in flists:
        newdf = pd.DataFrame()
        for pp in participants:
            if f == 'aln':
                df = aligned_flist[pp].to_data_frame()
            elif f == 'earlyrot':
                df = early_rot_flist[pp].to_data_frame()
            elif f == 'laterot':
                df = late_rot_flist[pp].to_data_frame()
            elif f == 'earlyrdm':
                df = early_rdm_flist[pp].to_data_frame()
            elif f == 'laterdm':
                df = late_rdm_flist[pp].to_data_frame()
            elif f == 'earlymir':
                df = early_mir_flist[pp].to_data_frame()
            elif f == 'latemir':
                df = late_mir_flist[pp].to_data_frame()
             
            time = df['time'].tolist()
            df = df[[channel]]
            ppdat = []
            for timept in range(0,800):
                subdf = df.loc[timept,:].mean()
                ppdat.append(subdf)
    
            newdf['pp%03d'% pp] = ppdat
    
        newdf['time'] = time
        newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s_%s_%s.csv' % (f, erps, channel))
        newdf.to_csv(newdf_filename)

In [ ]:
# Difference waves between aligned and perturbed conditions for C3
channels = ['C3']
#channels = ['C4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               early_rot = early_rot_flist,
               late_rot = late_rot_flist, 
               early_rdm = early_rdm_flist, 
               late_rdm = late_rdm_flist,
               early_mir = early_mir_flist,
               late_mir = late_mir_flist)
#plot
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine = 'mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

In [ ]:
#calculate difference waves for LRP C3
diffconds = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']
earlyrot_diff = []
laterot_diff = []
earlyrdm_diff = []
laterdm_diff = []
earlymir_diff = []
latemir_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rot'][pp]], 
                                                            weights=[-1, 1])
            earlyrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rot'][pp]], 
                                                         weights=[-1, 1])
            laterot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_rdm'][pp]], 
                                                         weights=[-1, 1])
            earlyrdm_diff.append(diffevks)
        elif cond == 3:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_rdm'][pp]], 
                                                         weights=[-1, 1])
            laterdm_diff.append(diffevks)
        elif cond == 4:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['early_mir'][pp]], 
                                                         weights=[-1, 1])
            earlymir_diff.append(diffevks)
        elif cond == 5:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['late_mir'][pp]], 
                                                         weights=[-1, 1])
            latemir_diff.append(diffevks)

In [ ]:
#saved difference waves as data frame for LRP C3

difflists = ['earlyrot', 'laterot', 'earlyrdm', 'laterdm', 'earlymir', 'latemir']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'earlyrot':
            df = earlyrot_diff[pp].to_data_frame()
        elif diff == 'laterot':
            df = laterot_diff[pp].to_data_frame()
        elif diff == 'earlyrdm':
            df = earlyrdm_diff[pp].to_data_frame()
        elif diff == 'laterdm':
            df = laterdm_diff[pp].to_data_frame()
        elif diff == 'earlymir':
            df = earlymir_diff[pp].to_data_frame()
        elif diff == 'latemir':
            df = latemir_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        #df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        df = df[['C3']]
        ppdat = []
        for timept in range(0,800):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_%s_%s_C3.csv' % (diff, erps))
    newdf.to_csv(newdf_filename)
        

In [ ]:
#plot difference waves for LRP C3 here in MNE
mne.viz.plot_compare_evokeds({'Earlyrot - Aligned':earlyrot_diff, 'Laterot - Aligned':laterot_diff, 'Earlyrdm - Aligned':earlyrdm_diff, 'Laterdm - Aligned':laterdm_diff, 'Earlymir - Aligned':earlymir_diff, 'Latemir - Aligned':latemir_diff}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');